In [1]:
#Mounting Google Drive from Google Colab
#from google.colab import drive
#drive.mount('/content/drive')

In [2]:
#Changing the current working directory to the Google Drive
#%cd /content/drive/My Drive/MLDL2024_project1-Enrico

In [3]:
#!pip install -U fvcore

In [4]:
#Importing the necessary libraries
import os
import torch
import numpy as np
from torchvision import transforms
from torch.utils.data import DataLoader
from datasets.gta5 import GTA5Custom
from datasets.cityscapes import CityscapesCustom
from models.bisenet.build_bisenet import BiSeNet
from models.discriminator import FCDiscriminator
from train_adversarial import train_adversarial
from utils import test_latency_FPS, test_FLOPs_params, plot_miou_over_epochs

In [5]:
#Set device agnostic code
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f'Using device: {device}')

#Set the manual seeds
torch.manual_seed(42)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(42)

#Set training parameters
gta5_height, gta5_width = (8, 16)
gta5_batch_size = 4

cityscapes_height, cityscapes_width = (8, 16)
cityscapes_batch_size = 4

n_epochs = 10

lambda_adv = 0.001

Using device: cpu


In [6]:
#Create Dataloaders for Cityscapes and GTA5
gta5_dir = os.path.dirname(os.getcwd()) + '/GTA5/GTA5/'
cityscapes_dir = os.path.dirname(os.getcwd()) + '/Cityscapes/Cityspaces/'

augment1 = transforms.ColorJitter(brightness=0.3, contrast=0.3, saturation=0.3, hue=0.3)
gta5_train_dataset_aug1 = GTA5Custom(gta5_dir, gta5_height, gta5_width, augment=augment1)
cityscapes_test_dataset = CityscapesCustom(cityscapes_dir, 'val', cityscapes_height, cityscapes_width)

gta5_train_dataloader_aug1 = DataLoader(gta5_train_dataset_aug1, gta5_batch_size, shuffle=True)
cityscapes_test_dataloader = DataLoader(cityscapes_test_dataset, cityscapes_batch_size, shuffle=False)

#Get the class names
class_names = cityscapes_test_dataset.get_class_names()

print(f'GTA5 (Train): {len(gta5_train_dataset_aug1)} images, divided into {len(gta5_train_dataloader_aug1)} batches of size {gta5_train_dataloader_aug1.batch_size}')
print(f'Cityscapes (Test): {len(cityscapes_test_dataset)} images, divided into {len(cityscapes_test_dataloader)} batches of size {cityscapes_test_dataloader.batch_size}')

GTA5 (Train): 2500 images, divided into 625 batches of size 4
Cityscapes (Test): 500 images, divided into 125 batches of size 4


In [ ]:
import torch
import time
import numpy as np
from tqdm import tqdm
from torch.nn import functional as F
from utils import poly_lr_scheduler, fast_hist, per_class_iou

def train_adversarial(gen, dis, g_criterion, d_criterion, g_optimizer, d_optimizer, lambda_adv, s_dataloader, t_dataloader, class_names, device, n_epochs, model_name):    
    n_classes = len(class_names)
    best_miou = 0.0
    best_class_iou = np.zeros(n_classes)
    best_epoch = 0
    all_train_miou = []
    all_test_miou = []

    #Initialize the labels for the adversarial training
    source_label = 0
    target_label = 1
    
    for epoch in range(n_epochs):

        start = time.time()      

        gen.train()
        dis.train()

        #train_hist = np.zeros((n_classes, n_classes))

        #Train G
        #Don't accumulate gradients in D
        for param in dis.parameters():
                param.requires_grad = False

        source_train_loop = tqdm(enumerate(s_dataloader), total=len(s_dataloader), leave=False)
        #Train G with source data
        for i, (inputs, labels) in source_train_loop:
            if i == 10:
                break
            inputs, labels = inputs.to(device), labels.to(device)

            g_optimizer.zero_grad()
            d_optimizer.zero_grad()
            
            pred1, _, _ = gen(inputs)
            seg_loss = g_criterion(pred1, labels)
            seg_loss.backward()

        target_train_loop = tqdm(enumerate(t_dataloader), total=len(t_dataloader), leave=False)
        #Train G with target data
        for i, (inputs, _) in target_train_loop:
            if i == 3:
                break
            inputs = inputs.to(device)

            pred_target1, _, _ = gen(inputs)
            d_out1 = dis(F.softmax(pred_target1, dim=1))#F.softmax(pred_target1, dim=1))
            adv_loss = d_criterion(d_out1, source_label)
            d_loss = lambda_adv * adv_loss
            d_loss.backward()

            #Train D

            #Bring back gradients in D
            for param in dis.parameters():
                param.requires_grad = True

            #Train D with source data
            d_out1 = dis(pred1.detach())#F.softmax(pred1, dim=1)
            d_loss = d_criterion(d_out1, source_label)
            d_loss.backward()

            #Train D with target data
            d_out1 = dis(pred_target1.detach())#F.softmax(pred_target1, dim=1)
            d_loss = d_criterion(d_out1, target_label)
            d_loss.backward()


            
            g_optimizer.step()
            d_optimizer.step()

            predictions = torch.argmax(outputs, dim=1)

            #train_hist += fast_hist(labels.cpu().numpy(), predictions.cpu().numpy(), n_classes)
            #train_loop.set_description(f'Epoch {epoch+1}/{n_epochs} (Train)')
            
        #train_class_iou = 100*per_class_iou(train_hist)
        #train_miou = np.mean(train_class_iou)
        #all_train_miou.append(train_miou)
        
        gen.eval()
        test_hist = np.zeros((n_classes, n_classes))
        test_loop = tqdm(enumerate(t_dataloader), total=len(t_dataloader), leave=False)
        with torch.no_grad():
            for i, (inputs, labels) in test_loop:
                inputs, labels = inputs.to(device), labels.to(device)
                
                outputs = gen(inputs)
                
                predictions = torch.argmax(outputs, dim=1)
                test_hist += fast_hist(labels.cpu().numpy(), predictions.cpu().numpy(), n_classes)
                test_loop.set_description(f'Epoch {epoch+1}/{n_epochs} (Test)')
                            
        test_class_iou = 100*per_class_iou(test_hist)
        test_miou = np.mean(test_class_iou)
        all_test_miou.append(test_miou)
        
        #Create a checkpoint dictionary
        checkpoint = {
            'epoch': epoch+1,
            'model_state_dict': gen.state_dict(),
            'optimizer_state_dict': g_optimizer.state_dict(),
            #'train_class_iou': train_class_iou,
            #'train_miou': train_miou,
            'test_class_iou': test_class_iou,
            'test_miou': test_miou,
        }

        #torch.save(checkpoint, f'checkpoints/{model_name}_checkpoint_epoch_{epoch+1}.pth')
        
        #Early stopping condition
        if test_miou > best_miou:
            best_miou = test_miou
            best_class_iou = test_class_iou
            best_epoch = epoch
            #torch.save(checkpoint, f'checkpoints/{model_name}_best_epoch_{epoch+1}.pth')

        end = time.time()

        #print(f'\nEpoch {epoch+1}/{n_epochs} [{(end-start) // 60:.0f}m {(end-start) % 60:.0f}s]: Train mIoU={train_miou:.2f}%, Test mIoU={test_miou:.2f}%')
        print(f'\nEpoch {epoch+1}/{n_epochs} [{(end-start) // 60:.0f}m {(end-start) % 60:.0f}s]: Test mIoU={test_miou:.2f}%')
        for class_name, iou in zip(class_names, test_class_iou):
            print(f'{class_name}: {iou:.2f}%', end=' ')

    print(f'\nBest mIoU={best_miou:.2f}% at epoch {best_epoch+1}')
    for class_name, iou in zip(class_names, best_class_iou):
        print(f'{class_name}: {iou:.2f}%', end=' ')

    return all_train_miou, all_test_miou, best_epoch

In [7]:
#Set up the segmentation network (our Generator) with the pretrained weights
generator_model = BiSeNet(num_classes=19, context_path='resnet18').to(device)

#Set up the loss function and the optimizer for the Generator
g_criterion = torch.nn.CrossEntropyLoss(ignore_index=255)
g_optimizer = torch.optim.SGD(generator_model.parameters(), lr=2.5e-2, momentum=0.9, weight_decay=1e-4)

In [8]:
#Set up the Discriminators
discriminator_model = FCDiscriminator(num_classes=19).to(device)

#Set up the loss functions and the optimizers for the Discriminators
d_criterion = torch.nn.BCEWithLogitsLoss()
d_optimizer = torch.optim.Adam(discriminator_model.parameters(), lr=1e-4, betas=(0.9, 0.99))

In [ ]:
all_train_miou, all_test_miou, best_epoch = train_adversarial(generator_model, discriminator_model,
                                                              g_criterion, d_criterion, g_optimizer, d_optimizer, lambda_adv,
                                                              gta5_train_dataloader_aug1, cityscapes_test_dataloader,
                                                              class_names, device, n_epochs, model_name='Adversarial')